In [1]:
import scipy.io as sio
from matplotlib import pyplot as plt 
%matplotlib inline

from mne.viz import plot_compare_evokeds

import numpy as np 

import os 
from andante_pd_ft2mne import import2mne  

#matfile = '/Users/nicolasfarrugia/Documents/recherche/PD/PDNewAnalysis/data/eeg_task/analysis/data_newfilt/probands/data_cleaned_newfilt_subj_01k101k1.mat'


Test on all subjects 
--

In [6]:
#datadir = '/Users/nicolasfarrugia/Documents/recherche/PD/PDNewAnalysis/data/eeg_task/analysis/data_newfilt/'
#datadir = '/Users/nicolasfarrugia/Documents/recherche/PD/PDNewAnalysis/data/eeg_task/analysis/data_ica_cleaned/'

datadir = '/home/brain/datasets/mpi_pd_cueing/data_ica_cleaned/'
resultdir = '/home/brain/datasets/mpi_pd_cueing/results/'

import os 

allcontrols = os.listdir(os.path.join(datadir,'probands'))
allpatients = os.listdir(os.path.join(datadir,'patients'))

In [3]:
from scipy.stats import mannwhitneyu

def mann_witt_matrix(mat,y):
    lcol = mat.shape[1]
    pvalue = np.zeros((lcol,lcol))
    statU = np.zeros((lcol,lcol))
    probav = np.zeros((2,lcol,lcol))
    unik = (np.unique(y))
    valstd=  unik[0]
    valdev= unik[1]
    for i in range(lcol):
        for j in range(lcol):
            #curp_S=mat[:,i,j,0]
            #curp_D=mat[:,i,j,1]
            curp_S_S=mat[(y==valstd),i,j,0]
            curp_S_D=mat[(y==valdev),i,j,0]
            
            stat,pval = mannwhitneyu(curp_S_S,curp_S_D,alternative='two-sided')
            statU[i,j]=stat
            pvalue[i,j]=pval
            
            probav[0,i,j]= np.mean(curp_S_S,axis=0)
            probav[1,i,j]= np.mean(curp_S_D,axis=0)
    
    return statU,pvalue,probav
            
def mann_witt_all(bigmat,y):
    resU = []
    resP = []
    resprobav = []
    for mat in bigmat:
        U,pval,probav = mann_witt_matrix(mat,y)
        
        resU.append(U)
        resP.append(pval)
        resprobav.append(probav)
        
    return np.stack(resU),np.stack(resP),np.stack(resprobav)

In [4]:
from andante_pd_ft2mne import import2mne  
from mne import Epochs,EpochsArray
from mne.channels import read_montage

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

import mne
from mne.datasets import sample
from mne.decoding import GeneralizingEstimator
from mne.decoding import CSP

from sklearn.model_selection import StratifiedKFold


def timegen_process_2x2(matfile,ncv=5,metric='accuracy',tmin=-0.05,tmax=0.15,
                        condnames=['iso_std','iso_dev'],condnames_2=['rnd_std','rnd_dev']):
    subjid = matfile[-8:-4]
    print("Subject : %s " % subjid)
    
    ## Open file 
    mneEpochs = import2mne(matfile)
    mneEpochs_short = mneEpochs.crop(tmin=tmin,tmax=tmax)
    
    
    montage = read_montage('standard_1020')

    mneEpochs_short.set_montage(montage)

    ## Perform Supervised Learning using Temporal Generalization 
    
    epochs = mneEpochs_short[condnames]
    epochs_cond2 = mneEpochs_short[condnames_2]
    
    clf = make_pipeline(StandardScaler(),LogisticRegression())

    time_gen = GeneralizingEstimator(clf, scoring=metric, n_jobs=-2)

    time_gen_cond2 = GeneralizingEstimator(clf, scoring=metric, n_jobs=-2)

    
    # Get the labels
    labels = epochs.events[:, -1]
    labels_2 = epochs_cond2.events[:, -1]

    # Cross validator
    cv = StratifiedKFold(n_splits=ncv, shuffle=True, random_state=42)

    
    ### We will calculate a 2x2 matrix
    ### First, let's cal
    
    
    scores1_1 = []
    scores1_2 = []
    scores2_1 = []
    scores2_2 = []
    
    proba1_1 = []
    proba1_2 = []
    proba2_1 = []
    proba2_2 = []
    
    U1_1 = []
    U1_2 = []
    U2_1 = []
    U2_2 = []
    
    allpval1_1=[]
    allpval1_2=[]
    allpval2_1=[]
    allpval2_2=[]

    
    for train, test in cv.split(epochs, labels):
        for train2, test2 in cv.split(epochs_cond2,labels_2):

            # Train classifier1 on train data of condition 1 
            time_gen.fit(X=epochs[train].get_data(), y=labels[train])

            # Train classifier2 on train data of condition 2 
            time_gen_cond2.fit(X=epochs_cond2[train2].get_data(), y=labels_2[train2])
            
            # Test Classifier1 on test data of condition 1
            scores1_1.append(time_gen.score(X=epochs[test].get_data(),y=labels[test]))
            
            U,allpval,proba_av = mann_witt_matrix(time_gen.predict_proba(X=epochs[test].get_data()),y=labels[test])
            proba1_1.append(proba_av)
            U1_1.append(U)
            allpval1_1.append(allpval)
            
            
            # Test Classifier1 on test data of condition 2
            scores1_2.append(time_gen.score(X=epochs_cond2[test2].get_data(),y=labels_2[test2]))
            
            U,allpval,proba_av = mann_witt_matrix(time_gen.predict_proba(X=epochs_cond2[test2].get_data()),y=labels_2[test2])
            proba1_2.append(proba_av)
            U1_2.append(U)
            allpval1_2.append(allpval)
            
            # Test Classifier2 on test data of condition 1
            scores2_1.append(time_gen_cond2.score(X=epochs[test].get_data(),y=labels[test]))
            
            U,allpval,proba_av = mann_witt_matrix(time_gen_cond2.predict_proba(X=epochs[test].get_data()),y=labels[test])
            proba2_1.append(proba_av)
            U2_1.append(U)
            allpval2_1.append(allpval)
            
            # Test Classifier2 on test data of condition 2
            scores2_2.append(time_gen_cond2.score(X=epochs_cond2[test2].get_data(),y=labels_2[test2]))
            
            U,allpval,proba_av = mann_witt_matrix(time_gen_cond2.predict_proba(X=epochs_cond2[test2].get_data()),y=labels_2[test2])
            proba2_2.append(proba_av)
            U2_2.append(U)
            allpval2_2.append(allpval)

        
    scores1_1 = np.stack(scores1_1)
    
    scores1_2 = np.stack(scores1_2)
    
    scores2_1 = np.stack(scores2_1)
    
    scores2_2 = np.stack(scores2_2)
    
    proba1_1 = np.stack(proba1_1)
    
    proba1_2 = np.stack(proba1_2)
    
    proba2_1 = np.stack(proba2_1)
    
    proba2_2 = np.stack(proba2_2)
    
    U1_1 = np.stack(U1_1)
    U1_2 = np.stack(U1_2)
    U2_1 = np.stack(U2_1)
    U2_2 = np.stack(U2_2)
    
    allpval1_1 = np.stack(allpval1_1)
    allpval1_2 = np.stack(allpval1_2)
    allpval2_1 = np.stack(allpval2_1)
    allpval2_2 = np.stack(allpval2_2)
    
    scores = np.stack([scores1_1,scores1_2,scores2_1,scores2_2])
    allU = np.stack([U1_1,U1_2,U2_1,U2_2])
    allpvals = np.stack([allpval1_1,allpval1_2,allpval2_1,allpval2_2])
    allproba = np.stack([proba1_1,proba1_2,proba2_1,proba2_2])
     
    return scores,allU,allpvals,allproba,epochs.times[[0, -1, 0, -1]]

Formal structure

In [ ]:
temporal = ['iso_std','rnd_std']
formal_iso = ['iso_std','iso_dev']
formal_rnd = ['rnd_std','rnd_dev']

tmin = -0.05
tmax = 0.52
ncv  = 2

#tmin = 0.25
#tmax = 0.40
#ncv  = 2

allscores_formal = []
allproba_formal = []

allU_formal = []
allpval_formal= []

listofsubj = [allcontrols,allpatients]

savenpz = True

for kk,group in enumerate(['probands','patients']):

    curlist = listofsubj[kk]
    
    for matfile in curlist[5:10]:
        curfile = os.path.join(datadir,group,matfile)
        subjid = curfile[-8:-4]

        allscores,allU,allpvals,allproba,timepoints= timegen_process_2x2(curfile,metric='roc_auc',tmin=tmin,tmax=tmax,
                                                 ncv=ncv,condnames=formal_iso,condnames_2=formal_rnd)
        
       

        if savenpz:
            np.savez_compressed(os.path.join(resultdir,"180802_%s_formal_conditionwise.npz" % subjid),
                                scores=allscores,
                                proba = allproba,
                                U = allU,
                                pval=allpvals)

Subject : 13k1 
633 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
